<a href="https://colab.research.google.com/github/zzjl20/tensorflow-machinelearning/blob/main/Chapter0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

本节与机器学习内容无关，纯粹是搭建运行环境。已经搭好的人可以略过本章。

# super computer的环境
首先登录超算的GPU节点：<pre>srum -p gpu-intr --gpus 1 --pty /bin/bash -i</pre>
查看当前架构:
<pre>arch
</pre>
返回结果：
> x86_64



# google colab的jupyter环境
注意，要在Edit -> Notebook Settings -> Hardware accelerator 中选择“GPU”.

获取当前python以及GPU的情况。

In [1]:
!python -V
!nvidia-smi

Python 3.11.13
Tue Jun 17 04:40:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+--------------------------------

在python环境中测试gpu环境。以及CUDA, cuDNN版本。

In [2]:
import tensorflow as tf
print(tf.__version__)
print("GPU 是否可用:", tf.config.list_physical_devices('GPU'))
print("编译时 CUDA 支持:", tf.sysconfig.get_build_info()['cuda_version'])
print("编译时 cuDNN 支持:", tf.sysconfig.get_build_info()['cudnn_version'])

from tensorflow.python.client import device_lib
devices = device_lib.list_local_devices()

2.18.0
GPU 是否可用: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
编译时 CUDA 支持: 12.5.1
编译时 cuDNN 支持: 9


测试一下，GPU是否真的投入了使用。

In [4]:
import tensorflow as tf
import timeit
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

with tf.device('/cpu:0'):
    cpu_a = tf.random.normal([10000, 1000])
    cpu_b = tf.random.normal([1000, 2000])
    print(cpu_a.device, cpu_b.device)

with tf.device('/gpu:0'):
    gpu_a = tf.random.normal([10000, 1000])
    gpu_b = tf.random.normal([1000, 2000])
    print(gpu_a.device, gpu_b.device)


def cpu_run():
    with tf.device('/cpu:0'):
        c = tf.matmul(cpu_a, cpu_b)
    return tf.reduce_sum(c).numpy()


def gpu_run():
    with tf.device('/gpu:0'):
        c = tf.matmul(gpu_a, gpu_b)
    return tf.reduce_sum(c).numpy()


# warm up
cpu_time = timeit.timeit(cpu_run, number=10)
gpu_time = timeit.timeit(gpu_run, number=10)
print('warmup:', cpu_time, gpu_time)

cpu_time = timeit.timeit(cpu_run, number=10)
gpu_time = timeit.timeit(gpu_run, number=10)
print('run time:', cpu_time, gpu_time)

/job:localhost/replica:0/task:0/device:CPU:0 /job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:GPU:0 /job:localhost/replica:0/task:0/device:GPU:0
warmup: 3.857086735999985 0.14755917800002294
run time: 4.946577391000005 0.1473261280000031


在实际执行结果中可看到，大约cpu执行时间是gpu执行时间的20～30倍。证明确实有GPU进入执行了程序。